In [1]:
import fantasy as fp
# noinspection PyUnresolvedReferences
from fantasyalgo import best_subset
import datetime as dt
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
data = pd.read_excel("19-01-2023.xlsx", index_col=0)

In [3]:
data.shape

(1172, 8)

In [4]:
event = fp.Event(6810)

In [5]:
players = event.get_players()
len(players)

80

In [6]:
costs_data = event.get_costs("costs_data.html")
len(costs_data)

80

In [7]:
for p in players:
    if p.name not in list(costs_data.keys()):
        name = f"{p.name}-"
        if name in list(costs_data.keys()):
            p.name = name
            print(p.name)

In [8]:
X = data.drop(columns=['player', 'event', 'pts']).to_numpy()
y = data['pts'].to_numpy()

In [9]:
costs = []
keys = list(costs_data.keys())
not_found = []

cnt = 0
for p in players:
    if p.name in keys:
        costs.append(costs_data[p.name])
    else:
        not_found.append(cnt)
    cnt += 1
len(costs)

80

In [10]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [12]:
predicted = []
rank = event.rank # get your event rank

for p in players:
    all_rating, all_wr = p.get_stats(dt.date.today() - dt.timedelta(days=90), dt.date.today(), fp.EventFilter.ALL)
    big_rating, big_wr = p.get_stats(dt.date.today() - dt.timedelta(days=90), dt.date.today(), fp.EventFilter.BIG)
    vector = np.array([[rank, all_rating, big_rating, all_wr, big_wr]])
    predicted.append(float(model.predict(vector)[0]))
len(predicted)

80

In [29]:
best = best_subset(predicted, costs)

In [30]:
names = [p.name for p in players]
pred_df = pd.DataFrame({'name': np.array(names), 'pred pts': np.array(predicted)})

In [31]:
pred_df['pred pts'] = pred_df['pred pts'].astype(float)
pred_df = pred_df.sort_values(by='pred pts', ascending=False)

In [32]:
pred_df.head(30)

,name,pred pts
8,sh1ro,136.206269
2,kscerato,123.555741
18,degster,72.199104
5,hobbit,70.898322
24,w0nderful,67.787518
1,yuurih,67.153017
29,sunpayus,62.827617
23,patsi,51.930660
43,staehr,50.112262
21,magixx,46.444214


In [33]:
for i in best:
    print(f"{players[i].name} --> {predicted[i]}")

kscerato --> 123.55574116508535
hobbit --> 70.89832166893422
neofrag --> 32.25571800683787
chopper --> 28.031288756892252
k1to --> 20.902137638593672
